In [ ]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()

model_package_group_name = f"BCAF-TrialModelPackageGroup" # Ganti sesuai kebutuhan, nama hanya untuk membedakan atau boleh menggunakan nama yang sebelumnya
pipeline_name = f"BCAF-TrialPipeline" # Ganti sesuai kebutuhan, nama hanya untuk membedakan atau boleh menggunakan nama yang sebelumnya

In [ ]:
# Ganti Training_CarPriceML2023 dengan nama folder untuk training
from pipelines.Training_CarPriceML2023.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name
)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()